In [1]:
import SWAN_file_generation
import pandas as pd
audio_parent_folder = "SWAN_audio"
sfg = SWAN_file_generation.SWAN_file_generation(audio_parent_folder)

pframe_audio_files = sfg.make_pandaframe_from_list_of_audio_files()

In [2]:
pframe_audio_files = sfg.append_audio_length_to_pframe(audio_parent_folder, pframe_audio_files)
#pframe_audio_files

In [3]:
languages = pframe_audio_files['language'].unique()
channels = pframe_audio_files['channel'].unique()

# All sessions, all languages

In [6]:
language_combinations = []
for language_enroll in languages:
    for language_test in languages:
        if language_enroll != language_test:
            if "English" not in language_enroll and "English" not in language_test:
                continue
        language_combinations.append("{}-{}".format(language_enroll, language_test))
language_combinations

# language_combinations_only_with_english = []
# for language_combination in language_combinations:
#     if "English" in language_combination:
#         language_combinations_only_with_english.append(language_combination)
        
# language_combinations_only_with_english

language_combinations_with_channels = []
for language_combination in language_combinations:
    for channel_enroll in channels:
        for channel_test in channels:
            if channel_enroll != "t" and channel_test != "t":
                language_combinations_with_channels.append("{}-{}-{}".format(language_combination, channel_enroll, channel_test))
language_combinations_with_channels

trials = []
for language_channel_comb in language_combinations_with_channels:
    for test_session in range(3, 7):
        trials.append("{}-02-0{}".format(language_channel_comb, test_session))
trials

['English-English-p-p-02-03',
 'English-English-p-p-02-04',
 'English-English-p-p-02-05',
 'English-English-p-p-02-06',
 'English-Norwegian-p-p-02-03',
 'English-Norwegian-p-p-02-04',
 'English-Norwegian-p-p-02-05',
 'English-Norwegian-p-p-02-06',
 'English-Hindi-p-p-02-03',
 'English-Hindi-p-p-02-04',
 'English-Hindi-p-p-02-05',
 'English-Hindi-p-p-02-06',
 'English-French-p-p-02-03',
 'English-French-p-p-02-04',
 'English-French-p-p-02-05',
 'English-French-p-p-02-06',
 'Norwegian-English-p-p-02-03',
 'Norwegian-English-p-p-02-04',
 'Norwegian-English-p-p-02-05',
 'Norwegian-English-p-p-02-06',
 'Norwegian-Norwegian-p-p-02-03',
 'Norwegian-Norwegian-p-p-02-04',
 'Norwegian-Norwegian-p-p-02-05',
 'Norwegian-Norwegian-p-p-02-06',
 'Hindi-English-p-p-02-03',
 'Hindi-English-p-p-02-04',
 'Hindi-English-p-p-02-05',
 'Hindi-English-p-p-02-06',
 'Hindi-Hindi-p-p-02-03',
 'Hindi-Hindi-p-p-02-04',
 'Hindi-Hindi-p-p-02-05',
 'Hindi-Hindi-p-p-02-06',
 'French-English-p-p-02-03',
 'French-Englis

In [13]:
import shutil
import os
folder_path = "SWAN_multiple"

if os.path.exists(folder_path):
            shutil.rmtree(folder_path)

os.mkdir(folder_path)

df_language_combinations = pd.DataFrame(columns=['Enrollment', 'Test', 'Channel_enroll', 'Channel_test', 'Session_enrollment', 'Session_test'])

for trial in trials:
   
    enroll_language, test_language, channel_enroll, channel_test, enrollment_session, test_session = trial.split("-")
    print("E-lang: {}, T-lang: {}, E-chan: {}, T-chan: {}, E-Sess: {}, T-Sess: {}".format(enroll_language, test_language, channel_enroll, channel_test, enrollment_session, test_session))
    df_language_combinations = df_language_combinations.append({'Enrollment': enroll_language, 'Test': test_language, 'Channel_enroll': channel_enroll, 'Channel_test' : channel_test, 'Session_enrollment' : enrollment_session, 'Session_test': test_session}, ignore_index=True)
    
    data_enrollment, data_test = sfg.split_data_into_enrollment_and_test(pframe_audio_files, enroll_language, test_language, channel_enroll, channel_test, enrollment_session, test_session)
    
    if test_session in ('05', '06'):
        print("Running session {}, removing subject 11 due to missing audio files".format(test_session))
        data_enrollment = data_enrollment[~data_enrollment['subject_id'].isin(["10011"])]
        data_test = data_test[~data_test['subject_id'].isin(["10011"])]
        print(data_enrollment['subject_id'].unique())
    
    #print("Pframe enrollment size: {}, test size {}".format(len(data_enrollment['subject_id'].unique()), len(data_test['subject_id'].unique())))
    subject_id_from_enrollment = data_enrollment['subject_id'].unique()
    print("Lan: {} - Test subjects before: {}".format(test_language, len(data_test['subject_id'].unique())))
    data_test = data_test[data_test['subject_id'].isin(subject_id_from_enrollment)]
    print("Lan:{} - Test subjects after: {}".format(test_language, len(data_test['subject_id'].unique())))
    
    
    
    pframe_enrollment_and_test = data_enrollment.append(data_test, ignore_index=True)    
    
    destination_folder = "{}/{}".format(folder_path, trial)
    sfg.create_folder_structure(destination_folder)
    output = sfg.create_metadata_files(pframe_enrollment_and_test, destination_folder)

    sfg.create_eval_files(data_enrollment, data_test, destination_folder)
    sfg.copy_files_to_enrollment_and_test(data_enrollment, data_test, destination_folder)
df_language_combinations.to_csv("{}/enrollment_test_channel_combinations.tsv".format(folder_path), sep = '\t', index=False)

E-lang: English, T-lang: English, E-chan: p, T-chan: p, E-Sess: 02, T-Sess: 03
Lan: English - Test subjects before: 195
Lan:English - Test subjects after: 195
E-lang: English, T-lang: English, E-chan: p, T-chan: p, E-Sess: 02, T-Sess: 04
Lan: English - Test subjects before: 195
Lan:English - Test subjects after: 195
E-lang: English, T-lang: English, E-chan: p, T-chan: p, E-Sess: 02, T-Sess: 05
Running session 05, removing subject 11 due to missing audio files
['10042' '10021' '10003' '10029' '10034' '10010' '10035' '10032' '10053'
 '10044' '10025' '10007' '10015' '10026' '10020' '10017' '10033' '10037'
 '10022' '10014' '10031' '10027' '10046' '10023' '10005' '10051' '10036'
 '10054' '10040' '10002' '10028' '10018' '10038' '10004' '10024' '10030'
 '10045' '10043' '10041' '10019' '10016' '10052' '10039' '10012' '10013'
 '10047' '10006' '10001' '10008' '60042' '60009' '60034' '60010' '60050'
 '60032' '60044' '60025' '60007' '60015' '60026' '60020' '60017' '60033'
 '60037' '60014' '60031' 

E-lang: English, T-lang: Hindi, E-chan: p, T-chan: p, E-Sess: 02, T-Sess: 06
Running session 06, removing subject 11 due to missing audio files
['10042' '10021' '10003' '10029' '10034' '10010' '10035' '10032' '10053'
 '10044' '10025' '10007' '10015' '10026' '10020' '10017' '10033' '10037'
 '10022' '10014' '10031' '10027' '10046' '10023' '10005' '10051' '10036'
 '10054' '10040' '10002' '10028' '10018' '10038' '10004' '10024' '10030'
 '10045' '10043' '10041' '10019' '10016' '10052' '10039' '10012' '10013'
 '10047' '10006' '10001' '10008' '60042' '60009' '60034' '60010' '60050'
 '60032' '60044' '60025' '60007' '60015' '60026' '60020' '60017' '60033'
 '60037' '60014' '60031' '60046' '60011' '60023' '60005' '60051' '60036'
 '60040' '60002' '60028' '60018' '60038' '60004' '60024' '60030' '60045'
 '60043' '60041' '60016' '60049' '60055' '60039' '60012' '60013' '60047'
 '60006' '60048' '60001' '60008' '20042' '20021' '20003' '20009' '20029'
 '20034' '20010' '20035' '20050' '20044' '20025' '200

E-lang: Norwegian, T-lang: Norwegian, E-chan: p, T-chan: p, E-Sess: 02, T-Sess: 04
Lan: Norwegian - Test subjects before: 100
Lan:Norwegian - Test subjects after: 100
E-lang: Norwegian, T-lang: Norwegian, E-chan: p, T-chan: p, E-Sess: 02, T-Sess: 05
Running session 05, removing subject 11 due to missing audio files
['10042' '10021' '10003' '10029' '10034' '10010' '10035' '10032' '10053'
 '10044' '10025' '10007' '10015' '10026' '10020' '10017' '10033' '10037'
 '10022' '10014' '10031' '10027' '10046' '10023' '10005' '10051' '10036'
 '10054' '10040' '10002' '10028' '10018' '10038' '10004' '10024' '10030'
 '10045' '10043' '10041' '10019' '10016' '10052' '10039' '10012' '10013'
 '10047' '10006' '10001' '10008' '20042' '20021' '20003' '20009' '20029'
 '20034' '20010' '20035' '20050' '20044' '20025' '20015' '20026' '20020'
 '20017' '20033' '20037' '20022' '20014' '20031' '20027' '20046' '20011'
 '20023' '20005' '20051' '20036' '20040' '20002' '20028' '20018' '20038'
 '20004' '20024' '20030' '

# Single session, all languages

In [ ]:
language_combinations = []
for language_enroll in languages:
    for language_test in languages:
        if language_enroll != language_test:
            language_combinations.append("{}-{}".format(language_enroll, language_test))
language_combinations

language_combinations_only_with_english = []
for language_combination in language_combinations:
    if "English" in language_combination:
        language_combinations_only_with_english.append(language_combination)
        
language_combinations_only_with_english

language_combinations_with_channels = []
for language_combination in language_combinations_only_with_english:
    for channel_enroll in channels:
        for channel_test in channels:
            if channel_enroll != "t" and channel_test != "t":
                language_combinations_with_channels.append("{}-{}-{}".format(language_combination, channel_enroll, channel_test))
language_combinations_with_channels

In [ ]:
import shutil
import os
folder_path = "SWAN_multiple"

if os.path.exists(folder_path):
            shutil.rmtree(folder_path)

os.mkdir(folder_path)

df_language_combinations = pd.DataFrame(columns=['Enrollment', 'Test', 'Channel_enroll', 'Channel_test', 'Session_enrollment', 'Session_test'])
enrollment_session = "02"
test_session = "05"

data_test_p = []
data_enrollment_p = []
output = []
for language_combination_with_channel in language_combinations_with_channels:
   
    enroll_language, test_language, channel_enroll, channel_test = language_combination_with_channel.split("-")
    print("E-lang: {}, T-lang: {}, E-chan: {}, T-chan: {}".format(enroll_language, test_language, channel_enroll, channel_test))
    df_language_combinations = df_language_combinations.append({'Enrollment': enroll_language, 'Test': test_language, 'Channel_enroll': channel_enroll, 'Channel_test' : channel_test, 'Session_enrollment' : enrollment_session, 'Session_test': test_session}, ignore_index=True)
    data_enrollment, data_test = sfg.split_data_into_enrollment_and_test(pframe_audio_files, enroll_language, test_language, channel_enroll, channel_test, enrollment_session, test_session)
    
    
    #print("Pframe enrollment size: {}, test size {}".format(len(data_enrollment['subject_id'].unique()), len(data_test['subject_id'].unique())))
    subject_id_from_enrollment = data_enrollment['subject_id'].unique()
    print("Lan: {} - Test subjects before: {}".format(test_language, len(data_test['subject_id'].unique())))
    data_test = data_test[data_test['subject_id'].isin(subject_id_from_enrollment)]
    print("Lan:{} - Test subjects after: {}".format(test_language, len(data_test['subject_id'].unique())))
    
    
    
    pframe_enrollment_and_test = data_enrollment.append(data_test, ignore_index=True)    
    
    destination_folder = "{}/{}".format(folder_path,language_combination_with_channel)
    sfg.create_folder_structure(destination_folder)
    output = sfg.create_metadata_files(pframe_enrollment_and_test, destination_folder)

    sfg.create_eval_files(data_enrollment, data_test, destination_folder)
    sfg.copy_files_to_enrollment_and_test(data_enrollment, data_test, destination_folder)
df_language_combinations.to_csv("{}/enrollment_test_channel_combinations.tsv".format(folder_path), sep = '\t', index=False)